# How to merge addresses and polling locations

In [1]:
# Important imports
import pandas as pd
import numpy as np

#### Read in the csv files and store them in a Pandas dataframe.

In [2]:
precinct_polling_list = pd.read_csv('precinct_polling_list.csv')
precinct_polling_list.name = 'Precinct Polling List'
addresses = pd.read_csv('addresses.csv')
addresses.name = 'Addresses'

#### View the data to make sure it loaded correctly.

In [3]:
print(precinct_polling_list.name, ':')
display(precinct_polling_list.head())
print(addresses.name, ':')
display(addresses.head())

Precinct Polling List :


,Location Name,Street,City,State/ZIP,Country,Precinct
0,Yorkside Pizza,288 York Street,New Haven,CT 06511,USA,CON-069
1,Library,301-399 South Boulevard Drive,Bainbridge,GA 39819,USA,GEO-062
2,Luxury Boston,150-151 Tremont Street,Boston,MA 02111,USA,MAS-111
3,Ingleside Village Pizza,2395 Ingleside Avenue,Macon,GA 31204,USA,GEO-041
4,Pizza House,1007 Merchant Street,Ambridge,PA 15003,USA,MAS-018


Addresses :


,Street,Apt,City,State,Zip,Precinct ID
0,7 BEACON BLVD,NaN,PEABODY,MA,01960-6303,025-090
1,1847 TOWER DR,NaN,STOUGHTON,WI,53589-3539,055-015
2,4628 GREEN VALLEY RD,NaN,FAIRFIELD,CA,94534-1368,006-009
3,14 HALET ST,NaN,PORTLAND,ME,04102-1734,023-089
4,1606 BLACKBURN HEIGHTS DR,NaN,SEWICKLEY,PA,15143-8626,042-018


Some of the data entries are incorrect. We need to fix them. Luckily, it is possible to infer what is wrong with them.

In [4]:
precinct_polling_list = pd.read_csv('precinct_polling_list.csv')
bad_data = precinct_polling_list.isna().any(axis=1) # Get rows where data didnt work
display(precinct_polling_list[bad_data])

,Location Name,Street,City,State/ZIP,Country,Precinct
6,Hearth Pizzeria,974 Great Plain Avenue Needham MA 02492,USA,MAS-006,NaN,NaN
13,Community Center,29 Godwin Avenue,Ridgewood NJ 07450,USA,NEWJ-000,NaN
23,Upper Crust Pizzeria,1782 Massachusetts Ave Cambridge MA 02140,USA,MAS-070,NaN,NaN
30,Ian's Pizza Milwaukee,2035 East North Avenue Milwaukee WI,53211,USA,WIS-067,NaN


In [5]:
# Hearth Pizzeria
precinct_polling_list.iloc[6][4:] = precinct_polling_list.iloc[6][2:4] # shift data over
precinct_polling_list.iloc[6]['State/ZIP'] = " ".join(precinct_polling_list.iloc[6]['Street'].split()[-2:])
precinct_polling_list.iloc[6]['City'] = precinct_polling_list.iloc[6]['Street'].split()[-3]
precinct_polling_list.iloc[6]['Street'] = " ".join(precinct_polling_list.iloc[6]['Street'].split()[:-3])

# Community Center
precinct_polling_list.iloc[13][4:] = precinct_polling_list.iloc[13][3:5] # shift data over
precinct_polling_list.iloc[13]['State/ZIP'] = " ".join(precinct_polling_list.iloc[13]['City'].split()[-2:])
precinct_polling_list.iloc[13]['City'] = precinct_polling_list.iloc[13]['City'].split()[0]

# Upper Crust Pizzeria (Same fn as Hearth)
precinct_polling_list.iloc[23][4:] = precinct_polling_list.iloc[23][2:4] # shift data over
precinct_polling_list.iloc[23]['State/ZIP'] = " ".join(precinct_polling_list.iloc[23]['Street'].split()[-2:])
precinct_polling_list.iloc[23]['City'] = precinct_polling_list.iloc[23]['Street'].split()[-3]
precinct_polling_list.iloc[23]['Street'] = " ".join(precinct_polling_list.iloc[23]['Street'].split()[:-3])

# Ian's Pizza Milwaukee
precinct_polling_list.iloc[30][4:] = precinct_polling_list.iloc[30][3:5] # shift data over
precinct_polling_list.iloc[30]['State/ZIP'] = " ".join([precinct_polling_list.iloc[30]['Street'].split()[-1],
                                                        precinct_polling_list.iloc[30]['City'].split()[-1]])
precinct_polling_list.iloc[30]['City'] = precinct_polling_list.iloc[30]['Street'].split()[-2]
precinct_polling_list.iloc[30]['Street'] = " ".join(precinct_polling_list.iloc[30]['Street'].split()[:-3])

Lastly, courthouse has an extra dash in the precinct code.

In [6]:
display(precinct_polling_list.iloc[[27]])

,Location Name,Street,City,State/ZIP,Country,Precinct
27,Courthouse,9600-9708 West Linebaugh Avenue,Westchase,FL 33626,USA,FLO--67


In [7]:
# Courthouse (extra dash)
precinct_polling_list['Precinct'] = precinct_polling_list['Precinct'].str.replace('--', '-0')

display(precinct_polling_list[bad_data])

,Location Name,Street,City,State/ZIP,Country,Precinct
6,Hearth Pizzeria,974 Great Plain Avenue,Needham,MA 02492,USA,MAS-006
13,Community Center,29 Godwin Avenue,Ridgewood,NJ 07450,USA,NEWJ-000
23,Upper Crust Pizzeria,1782 Massachusetts Ave,Cambridge,MA 02140,USA,MAS-070
30,Ian's Pizza Milwaukee,2035 East North,Milwaukee,WI 53211,USA,WIS-067


#### To make it easier to combine the tables, we can use the state fips codes

In [8]:
state_fips = pd.read_csv('state_fips.csv').set_index('stusps').astype(str)
state_fips.name = 'State Fips'
print(state_fips.name, ':')
display(state_fips.head())

State Fips :


,stname,st
stusps,,
AL,Alabama,1
AK,Alaska,2
AZ,Arizona,4
AR,Arkansas,5
CA,California,6


Get the state and zip from the existing column.

In [9]:
precinct_polling_list['State'] = precinct_polling_list['State/ZIP'].str.split(expand=True)[0]
precinct_polling_list['ZIP'] = precinct_polling_list['State/ZIP'].str.split(expand=True)[1]
del(precinct_polling_list['State/ZIP'])

Join the state fips table with this for an accurate fips number.

In [10]:
precinct_polling_list = precinct_polling_list.join(state_fips, on='State')

Create a precinct id with state fips and precinct id to match the addresses table, then set it as index so we can join it later.

In [11]:
precinct_polling_list['Precinct ID'] = precinct_polling_list['st'].str.zfill(3) + '-' + precinct_polling_list['Precinct'].str.split('-', expand=True)[1]
precinct_polling_list = precinct_polling_list.set_index('Precinct ID')
display(precinct_polling_list.head())

,Location Name,Street,City,Country,Precinct,State,ZIP,stname,st
Precinct ID,,,,,,,,,
009-069,Yorkside Pizza,288 York Street,New Haven,USA,CON-069,CT,06511,Connecticut,9
013-062,Library,301-399 South Boulevard Drive,Bainbridge,USA,GEO-062,GA,39819,Georgia,13
025-111,Luxury Boston,150-151 Tremont Street,Boston,USA,MAS-111,MA,02111,Massachusetts,25
013-041,Ingleside Village Pizza,2395 Ingleside Avenue,Macon,USA,GEO-041,GA,31204,Georgia,13
042-018,Pizza House,1007 Merchant Street,Ambridge,USA,MAS-018,PA,15003,Pennsylvania,42


Join addresses table with the polling list using the precinct id we created.

In [12]:
addresses_temp = addresses.join(precinct_polling_list[['Location Name', 'Street', 'City']], on='Precinct ID', rsuffix ='_precinct')
display(addresses_temp.head())

,Street,Apt,City,State,Zip,Precinct ID,Location Name,Street_precinct,City_precinct
0,7 BEACON BLVD,NaN,PEABODY,MA,01960-6303,025-090,Stonewood Tavern,139 Lynnfield Street,Peabody
1,1847 TOWER DR,NaN,STOUGHTON,WI,53589-3539,055-015,Michelangelo's Coffee House,114 State Street,Madison
2,4628 GREEN VALLEY RD,NaN,FAIRFIELD,CA,94534-1368,006-009,Mary's Pizza Shack,3085 Jefferson Street,Napa
3,14 HALET ST,NaN,PORTLAND,ME,04102-1734,023-089,Elementaray School Auditorium,574 Congress Street,Portland
4,1606 BLACKBURN HEIGHTS DR,NaN,SEWICKLEY,PA,15143-8626,042-018,Pizza House,1007 Merchant Street,Ambridge


There are some NaNs in the join. This means there were some precinct IDs that we did not have in our table. Ex:

In [13]:
display(addresses_temp.iloc[[12]])

,Street,Apt,City,State,Zip,Precinct ID,Location Name,Street_precinct,City_precinct
12,2 Gunderson Road,NaN,WILMINGTON,MA,01887-1520,048-010,NaN,NaN,NaN


The MA fips is 25, not 48. We can construct our own Precinct ID if we trust the state field more than the precinct ID field.

In [14]:
addresses = addresses.join(state_fips, on='State')
addresses['Precinct ID'] = addresses['st'].str.zfill(3) + '-' + addresses['Precinct ID'].str.split('-', expand=True)[1]
addresses = addresses.join(precinct_polling_list[['Location Name', 'Street', 'City']], on='Precinct ID', rsuffix ='_precinct')
display(addresses.head())

,Street,Apt,City,State,Zip,Precinct ID,stname,st,Location Name,Street_precinct,City_precinct
0,7 BEACON BLVD,NaN,PEABODY,MA,01960-6303,025-090,Massachusetts,25,Stonewood Tavern,139 Lynnfield Street,Peabody
1,1847 TOWER DR,NaN,STOUGHTON,WI,53589-3539,055-015,Wisconsin,55,Michelangelo's Coffee House,114 State Street,Madison
2,4628 GREEN VALLEY RD,NaN,FAIRFIELD,CA,94534-1368,006-009,California,6,Mary's Pizza Shack,3085 Jefferson Street,Napa
3,14 HALET ST,NaN,PORTLAND,ME,04102-1734,023-089,Maine,23,Elementaray School Auditorium,574 Congress Street,Portland
4,1606 BLACKBURN HEIGHTS DR,NaN,SEWICKLEY,PA,15143-8626,042-018,Pennsylvania,42,Pizza House,1007 Merchant Street,Ambridge


There are other values which we could not find a precinct code for. It looks like we do not have that polling location, so we can leave the value as NaN.

#### We are done!

In [15]:
addresses.to_csv('addresses_with_polling.csv', index=False)

## Differences with more table entries

This would only be slightly harder if the tables were bigger. The basic strategy, to create a common precinct ID so that we can join the tables together, is the same, but the cleaning of the data becomes a lengthy task. Luckily, we can use the pandas `isna()` function to sort the dataframe by bad values to ease in cleaning. In the methods used above, I used python to fix the data, when it would have probably been easier to just add a few commas to the csv file.

***
# Problem 2 (Creating VIP Files)

## polling_location.txt

address_location_name,address_line1,address_line2,address_line3,address_city,address_state,address_zip,directions,polling_hours,photo_url,id

In [16]:
precinct_polling_list.head()

,Location Name,Street,City,Country,Precinct,State,ZIP,stname,st
Precinct ID,,,,,,,,,
009-069,Yorkside Pizza,288 York Street,New Haven,USA,CON-069,CT,06511,Connecticut,9
013-062,Library,301-399 South Boulevard Drive,Bainbridge,USA,GEO-062,GA,39819,Georgia,13
025-111,Luxury Boston,150-151 Tremont Street,Boston,USA,MAS-111,MA,02111,Massachusetts,25
013-041,Ingleside Village Pizza,2395 Ingleside Avenue,Macon,USA,GEO-041,GA,31204,Georgia,13
042-018,Pizza House,1007 Merchant Street,Ambridge,USA,MAS-018,PA,15003,Pennsylvania,42


In [17]:
polling_location = precinct_polling_list.sort_index().reset_index()
polling_location['id'] = polling_location.index

precinct_polling_location = polling_location[['id', 'Precinct ID']]

polling_location['address_line2'] = np.NaN
polling_location['address_line3'] = np.NaN
polling_location['directions'] = np.NaN
polling_location['polling_hours'] = np.NaN
polling_location['photo_url'] = np.NaN

polling_location = polling_location[['Location Name', 'Street', 'address_line2', 'address_line3',
                                     'City', 'State', 'ZIP', 'directions', 'polling_hours',
                                     'photo_url', 'id']]
polling_location.columns = ['address_location_name', 'address_line1', 'address_line2', 'address_line3',
                            'address_city', 'address_state', 'address_zip', 'directions', 'polling_hours',
                            'photo_url', 'id']

display(polling_location.head())
polling_location.to_csv('polling_location.txt', index=False)

,address_location_name,address_line1,address_line2,address_line3,address_city,address_state,address_zip,directions,polling_hours,photo_url,id
0,Organ Stop Pizza,1149 East Southern Avenue,NaN,NaN,Mesa,AZ,85204,NaN,NaN,NaN,0
1,The Pruneyard Shopping Center,1875 South Bascom Avenue,NaN,NaN,Campbell,CA,95008,NaN,NaN,NaN,1
2,Mary's Pizza Shack,3085 Jefferson Street,NaN,NaN,Napa,CA,94559,NaN,NaN,NaN,2
3,Higgins Building,108 West 2nd Street,NaN,NaN,Los Angeles,CA,90012,NaN,NaN,NaN,3
4,Senior High School,563 Carlsbad Village Drive,NaN,NaN,Carlsbad,CA,92008,NaN,NaN,NaN,4


## precinct.txt

name,number,locality_id,ward,mail_only,ballot_style_image_url,id

In [18]:
precinct = precinct_polling_location[['Precinct ID']]
#precinct['name'] = range(60, 60 + len(precinct))
precinct['number'] = np.NaN
precinct['locality_id'] = range(3000, 3000 + len(precinct))
precinct['ward'] = np.NaN
precinct['mail_only'] = np.NaN
precinct['ballot_style_image_url'] = np.NaN
precinct['id'] = range(900100, 900100 + len(precinct))
precinct.columns = ['name'] + list(precinct.columns[1:])
precinct_polling_location['Precinct ID'] = precinct['id']
display(precinct.head())
precinct.to_csv('precinct.txt', index=False)

,name,number,locality_id,ward,mail_only,ballot_style_image_url,id
0,004-067,NaN,3000,NaN,NaN,NaN,900100
1,006-000,NaN,3001,NaN,NaN,NaN,900101
2,006-009,NaN,3002,NaN,NaN,NaN,900102
3,006-021,NaN,3003,NaN,NaN,NaN,900103
4,006-032,NaN,3004,NaN,NaN,NaN,900104


## precinct_polling_location.txt

In [19]:
precinct_polling_location.columns = ['polling_location_id', 'precinct_id']
precinct_polling_location = precinct_polling_location[['precinct_id', 'polling_location_id']]
display(precinct_polling_location.head())
precinct_polling_location.to_csv('precinct_polling_location.txt', index=False)

,precinct_id,polling_location_id
0,900100,0
1,900101,1
2,900102,2
3,900103,3
4,900104,4
